# DIGG forum rapport

* [#1482](https://github.com/salgo60/DIGG-skuggbacklog/issues/1482)
* Denna Notebook [IA DIGG forum](https://github.com/salgo60/DIGG-skuggbacklog/blob/master/Notebook/IA_DIGG%20forum.ipynb)
   * se även Notebook [1478 DIGG forum](https://github.com/salgo60/DIGG-skuggbacklog/blob/master/Notebook/1478%20DIGG%20forum.ipynb)



In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-12-08 19:36:44


In [2]:
import csv
import os

LOGFILE = "archive_log.csv"

LOG_FIELDS = [
    "pid",
    "url",
    "last_capture",
    "skipped_existing",
    "save_success",
    "save_location",
    "error"
]

# Create log file if not exists
if not os.path.exists(LOGFILE):
    with open(LOGFILE, "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=LOG_FIELDS)
        writer.writeheader()


In [3]:
processed_pids = set()

if os.path.exists(LOGFILE):
    import pandas as pd
    logdf = pd.read_csv(LOGFILE)
    processed_pids = set(logdf["pid"].dropna().astype(int).tolist())

processed_pids = set(processed_pids)
print("Resume mode: already processed", len(processed_pids), "PIDs")


Resume mode: already processed 757 PIDs


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 5)

# 1. läs in
df = pd.read_json("forum_data.json")

# Kolla snabbt
df.head(), df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4063 entries, 0 to 4062
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tid              4063 non-null   int64 
 1   cid              4063 non-null   int64 
 2   slug             4063 non-null   object
 3   topic_title      4063 non-null   object
 4   topic_timestamp  4063 non-null   object
 5   pid              4063 non-null   int64 
 6   post_index       4063 non-null   int64 
 7   post_timestamp   4063 non-null   object
 8   uid              4063 non-null   int64 
 9   content_html     4063 non-null   object
 10  content_len      4063 non-null   int64 
 11  post_date        4063 non-null   int64 
 12  username         3225 non-null   object
 13  username_norm    4063 non-null   object
 14  is_digg          4063 non-null   bool  
 15  post_month       4063 non-null   object
dtypes: bool(1), int64(7), object(8)
memory usage: 480.2+ KB


(    tid  cid                                               slug  \
 0   282   23  282/hjälp-nosad-och-dataportalens-communityfor...   
 1  1059   23  1059/dataportal-se-har-stöd-för-sparql-queries...   
 2  1059   23  1059/dataportal-se-har-stöd-för-sparql-queries...   
 3  1059   23  1059/dataportal-se-har-stöd-för-sparql-queries...   
 4  1059   23  1059/dataportal-se-har-stöd-för-sparql-queries...   
 
                                          topic_title  \
 0  Hjälp NOSAD och dataportalens communityforum a...   
 1  Dataportal.se har stöd för Sparql-queries, vis...   
 2  Dataportal.se har stöd för Sparql-queries, vis...   
 3  Dataportal.se har stöd för Sparql-queries, vis...   
 4  Dataportal.se har stöd för Sparql-queries, vis...   
 
             topic_timestamp   pid  post_index            post_timestamp   uid  \
 0  2022-01-11T17:30:39.030Z  1213           0  2022-01-11T17:30:39.030Z    62   
 1  2025-10-06T09:12:42.687Z  5304           0  2025-10-06T09:12:42.687Z   127   


In [5]:
df.head(3)



,tid,cid,slug,topic_title,topic_timestamp,pid,post_index,post_timestamp,uid,content_html,content_len,post_date,username,username_norm,is_digg,post_month
0,282,23,282/hjälp-nosad-och-dataportalens-communityfor...,Hjälp NOSAD och dataportalens communityforum a...,2022-01-11T17:30:39.030Z,1213,0,2022-01-11T17:30:39.030Z,62,"<p dir=""auto"">11 januari hade NOSAD (Network o...",5170,1641922239030,Maria_Dalhage,maria_dalhage,True,2022-01
1,1059,23,1059/dataportal-se-har-stöd-för-sparql-queries...,"Dataportal.se har stöd för Sparql-queries, vis...",2025-10-06T09:12:42.687Z,5304,0,2025-10-06T09:12:42.687Z,127,"<p dir=""auto"">Hej tänkte bara meddela att en f...",4756,1759741962687,tomasmonsen,tomasmonsen,False,2025-10
2,1059,23,1059/dataportal-se-har-stöd-för-sparql-queries...,"Dataportal.se har stöd för Sparql-queries, vis...",2025-10-06T09:12:42.687Z,5305,1,2025-10-06T10:06:41.301Z,1802,"<p dir=""auto"">Trevligt man kan undra varför de...",557,1759745201301,Magnus Sälgö,magnus sälgö,False,2025-10


In [6]:
from concurrent.futures import ThreadPoolExecutor, as_completed

POOL = ThreadPoolExecutor(max_workers=5)  # safe, metadata only

import requests
import time
from requests.exceptions import RequestException

BASE_URL = "https://community.dataportal.se/topic/"
SAVE_API = "https://web.archive.org/save/"
HEADERS = {"User-Agent": "ArchiveBot/1.0 (salgo60@msn.com)"}

session = requests.Session()
session.headers.update(HEADERS)

def write_log(row):
    with open(LOGFILE, "a", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=LOG_FIELDS)
        writer.writerow(row)
import requests

def wayback_status(url):
    api = "https://archive.org/wayback/available?url=" + url
    try:
        r = session.get(api, timeout=30)
        j = r.json()

        snap = j.get("archived_snapshots", {}).get("closest")

        if snap:
            return {
                "available": snap.get("available", False),
                "timestamp": snap.get("timestamp"),
                "snapshot_url": snap.get("url")
            }
        return {"available": False, "timestamp": None, "snapshot_url": None}

    except Exception as e:
        print("⚠ Wayback API error:", e)
        return {"available": False, "timestamp": None, "snapshot_url": None}
import threading
import time
import random

def process_pid(pid):
    base_url = f"{BASE_URL}{pid}"

    result = {
        "pid": pid,
        "url": None,
        "last_capture": None,
        "skipped_existing": False,
        "save_success": False,
        "save_location": None,
        "error": None
    }

    # 1. Resolve URL
    final_url = resolve_final_url(base_url)
    if not final_url:
        result["error"] = "Cannot resolve URL"
        write_log(result)
        return result

    result["url"] = final_url

    # 2. Check if source exists
    if not url_exists(final_url):
        result["error"] = "Source URL missing"
        write_log(result)
        return result

    # 3. Check wayback availability
    wb = wayback_status(final_url)
    result["last_capture"] = wb["timestamp"]

    # Skip if capture < 7 days old
    if wb["timestamp"]:
        result["skipped_existing"] = True
        write_log(result)
        return result

    # 4. Save (rate-limited)
    success, loc = save_to_wayback(final_url)
    result["save_success"] = success
    result["save_location"] = loc
    if not success:
        result["error"] = loc

    write_log(result)
    return result

save_lock = threading.Lock()

def wayback_status(url, retries=3):
    api = "https://archive.org/wayback/available?url=" + url

    for attempt in range(retries):
        try:
            r = session.get(api, timeout=30)
            if r.status_code != 200:
                time.sleep(1 + attempt)
                continue

            # Try to parse JSON safely
            try:
                j = r.json()
            except ValueError:
                # IA returned HTML or empty content → retry
                time.sleep(1 + attempt)
                continue

            snap = j.get("archived_snapshots", {}).get("closest")
            if snap:
                return {
                    "available": snap.get("available", False),
                    "timestamp": snap.get("timestamp"),
                    "snapshot_url": snap.get("url")
                }
            return {"available": False, "timestamp": None, "snapshot_url": None}

        except Exception:
            time.sleep(1 + attempt)

    # Fallback: treat as "no capture"
    return {"available": False, "timestamp": None, "snapshot_url": None}




def resolve_final_url(url):
    """Follow redirects and return the final URL (with slug if present)."""
    try:
        r = session.get(url, timeout=20, allow_redirects=True)
        if r.status_code == 200:
            return r.url
        print(f"  ⚠ Cannot load {url}: {r.status_code}")
        return None
    except RequestException as e:
        print(f"  ⚠ Error resolving {url}: {e}")
        return None

def url_exists(url, retries=3):
    """Return True only if the source URL returns 200."""
    for attempt in range(1, retries + 1):
        try:
            r = session.head(url, timeout=20, allow_redirects=True)
            if r.status_code == 200:
                return True
            elif r.status_code >= 400:
                print(f"  ⚠ Source returned {r.status_code} → skip")
                return False
        except RequestException as e:
            print(f"  ⚠ HEAD error: {e} (attempt {attempt})")
            time.sleep(2 * attempt)

    print("  ❌ Could not verify URL → skip")
    return False


def archive_url(url, max_retries=8):
    delay = 20  # start slow
    for attempt in range(1, max_retries + 1):
        print(f"  Attempt {attempt}: archiving {url}")

        try:
            r = session.post(
                SAVE_API,
                data={"url": url},
                timeout=180,
                allow_redirects=True
            )

            # Already archived
            if r.status_code in (200, 301, 302):
                loc = r.headers.get("Content-Location")
                if loc:
                    print("    ✔ Archived at:", "https://web.archive.org" + loc)
                else:
                    print("    ✔ Archived (no location header)")
                return True

            if r.status_code == 429:
                retry_after = r.headers.get("Retry-After", 60)
                print(f"    ⚠ 429 rate limit → sleeping {retry_after}s")
                time.sleep(int(retry_after))
                continue

            print(f"    ✖ Status {r.status_code}, retrying...")

        except RequestException as e:
            print(f"    ✖ Error: {e}, retrying...")

        # exponential backoff + jitter
        sleep_time = delay * (1 + random.random())
        print(f"    Sleeping {sleep_time:.1f}s...")
        time.sleep(sleep_time)
        delay *= 2

    print("    ❌ Failed after all retries:", url)
    return False


# ---- MAIN LOOP ---- 
df_sorted = df.sort_values("pid", ascending=False)
todo = df_sorted[df_sorted["pid"] <= 950]
todo = todo[~todo["pid"].isin(processed_pids)]

print("Will process", len(todo), "remaining PIDs")

futures = {POOL.submit(process_pid, int(row.pid)): row.pid for row in todo.itertuples()}

for future in as_completed(futures):
    pid = futures[future]
    try:
        res = future.result()
        print(f"[PID {pid}] done → last capture {res['last_capture']}, saved {res['save_success']}")
    except Exception as e:
        print(f"[PID {pid}] error:", e)

    import random

    time.sleep(random.uniform(20, 60))   # instead of 5 seconds

import pandas as pd

logdf = pd.read_csv(LOGFILE)
display(logdf.tail(100))


Will process 0 remaining PIDs


,pid,url,last_capture,skipped_existing,save_success,save_location,error
997,658,https://community.dataportal.se/topic/658/prot...,2.025032e+13,True,False,NaN,NaN
998,657,https://community.dataportal.se/topic/657/fund...,2.025082e+13,True,False,NaN,NaN
999,656,https://community.dataportal.se/topic/656/digg...,2.025043e+13,True,False,NaN,NaN
1000,653,NaN,NaN,False,False,NaN,Cannot resolve URL
1001,655,https://community.dataportal.se/topic/655/nati...,2.025043e+13,True,False,NaN,NaN
1002,654,https://community.dataportal.se/topic/654/ny-s...,2.024111e+13,True,False,NaN,NaN
1003,650,https://community.dataportal.se/topic/650/nati...,2.025091e+13,True,False,NaN,NaN
1004,651,https://community.dataportal.se/topic/651/data...,2.025071e+13,True,False,NaN,NaN
1005,649,https://community.dataportal.se/topic/649/samm...,2.025043e+13,True,False,NaN,NaN
1006,648,https://community.dataportal.se/topic/648/guid...,2.025043e+13,True,False,NaN,NaN


In [8]:
display(logdf)

,pid,url,last_capture,skipped_existing,save_success,save_location,error
0,948,https://community.dataportal.se/topic/948/bidr...,2.025091e+13,True,False,NaN,NaN
1,947,https://community.dataportal.se/topic/947/den-...,2.025091e+13,True,False,NaN,NaN
2,950,https://community.dataportal.se/topic/950/data...,2.025081e+13,True,False,NaN,NaN
3,944,https://community.dataportal.se/topic/944/om-d...,2.025033e+13,True,False,NaN,NaN
4,943,https://community.dataportal.se/topic/943/om-d...,2.025051e+13,True,False,NaN,NaN
...,...,...,...,...,...,...,...
1012,639,NaN,NaN,False,False,NaN,Cannot resolve URL
1013,638,NaN,NaN,False,False,NaN,Cannot resolve URL
1014,678,https://community.dataportal.se/topic/678/samn...,NaN,False,True,NaN,NaN
1015,645,https://community.dataportal.se/topic/645/sekr...,NaN,False,True,NaN,NaN


In [7]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-12-08 19:36:45
Total time elapsed: 00 minutes 00.69 seconds
